In [ ]:
import os
import json
from pymongo import MongoClient

In [ ]:
# min / sec
client = MongoClient('127.0.0.1', 27017)
db = client['accumulated_putinmissing'] #Create DB

In [ ]:
root = './putinmissing'
for folders in os.listdir(root):
    folder = root +'/'+folders
    #print(fol)
    col = db[folders]
    for fol in os.listdir(folder):
        pth = folder+'/'+fol
        if fol.startswith("retweets") and fol.endswith(".json") :
            for line in open(os.path.join(pth)):
                #print("###")
                rt = json.loads(line)
                col.insert_one(rt)

In [ ]:
for col_name in db.list_collection_names():
    col = db.get_collection(col_name)
    for x in col.find():
       col.update_many({},[
            { '$set': 
                { 'retweet_date': 
                    {
                        '$dateToString': {
                            'format': '%Y-%m-%dT%H:%M', 
                            "date":{
                                '$toDate':'$created_at'
                            }
                        }
                     }  
                }
            }
        ])

In [ ]:
#AGGREGATION
for col_name in db.list_collection_names():
    col = db.get_collection(col_name)
    result = col.aggregate([
    {
        '$addFields': { #SHAPE created_at to specified format
            'date': {
                '$dateToString': {
                    'format': '%Y-%m-%dT%H:%M', 
                    'date': {
                        '$toDate': '$created_at'
                    }
                }
            }
        }
    }, {
        '$group': { # group the same date and time (minute) and count freq of them
            '_id': '$date', 
            'count': {
                '$sum': 1
            }
        }
    }, { # ascending the days
        '$sort': {
            '_id': 1
        }
    }, {
        '$group': { # pushing days and count from object to array
            '_id': None, 
            'days': {
                
                '$push': '$_id'
            }, 
            'count': {
                '$push': '$count'
            }
        }
    }, {
        '$project': {
            'days': { # put both array (days and count) together within days Array 
                '$map': {
                    'input': {
                        '$range': [
                            0, {'$size': '$days'}]}, 
                    'in': {
                        '_id': { #_id as date and time
                            '$arrayElemAt': ['$days', '$$this']}, 
                        'amount': { # amount as total accumulative 
                            '$sum': {'$slice': ['$count', 0, {'$add': ['$$this', 1]}]}}, # slice them from array and sum them together
                        'freq': { # freq as frequencies per min in hour
                            '$sum': {'$slice': ['$count', '$$this', 1]}} # slice them from array and sum them together
                    }
                }
            }
        }
    }, {
        '$unwind': '$days'
    }, {
        '$replaceRoot': {
            'newRoot': '$days'
        }
    }, {
        '$sort': {
            '_id': 1
        }
    }
])
    for data in result:
        query = { "retweet_date" : data["_id"]}
        update = {
            "$set":{
                "amount":data["amount"],
                "freq": data['freq']
            }
        }
        col.update_many(query, update)
       

In [13]:
# JUST PRINT RESULT FOR CHECKING 
for col_name in db.list_collection_names():
    col = db.get_collection(col_name)
    for x in col.find():
        print(col.name, x['retweet_date'],"amount : ", x['amount'], "freq : ", x['freq'])
    print("==============================")

576513463738109954 2015-03-13T22:42 amount :  3 freq :  3
576513463738109954 2015-03-13T22:42 amount :  3 freq :  3
576513463738109954 2015-03-13T22:42 amount :  3 freq :  3
576513463738109954 2015-03-13T22:43 amount :  6 freq :  3
576513463738109954 2015-03-13T22:43 amount :  6 freq :  3
576513463738109954 2015-03-13T22:43 amount :  6 freq :  3
576513463738109954 2015-03-13T22:44 amount :  10 freq :  4
576513463738109954 2015-03-13T22:44 amount :  10 freq :  4
576513463738109954 2015-03-13T22:44 amount :  10 freq :  4
576513463738109954 2015-03-13T22:44 amount :  10 freq :  4
576513463738109954 2015-03-13T22:45 amount :  12 freq :  2
576513463738109954 2015-03-13T22:45 amount :  12 freq :  2
576513463738109954 2015-03-13T22:47 amount :  13 freq :  1
576513463738109954 2015-03-13T22:49 amount :  14 freq :  1
576513463738109954 2015-03-13T22:51 amount :  15 freq :  1
576513463738109954 2015-03-13T22:52 amount :  16 freq :  1
576513463738109954 2015-03-13T22:57 amount :  17 freq :  1
576